# EXAMPLE 1

This is an example to start using REWET. We go step by step through an example city, with imaginary restoration crew and scenario.

## Setup

First, let's setup the enviroment.

### Setup Drive

The example data are uplaoded in the Google drive. You can make a shortcut fromn the [REWET examples shared drive](https://drive.google.com/drive/folders/1fmhidiooBYw_hDtB9XITSvzl3Iaa9IOq?usp=drive_link) to the root of your Google drive. Then you mount your shared drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Then, lest's make a folder named *Result* in the Google Drive

### Install Depencencies

Now we need to to install depencies such as REWET. You can install REWET easily by running pip command: pip3 install REWET.

Note that in Google Collab, you there may be some error or warnnings regarding compatibility of the depencies. Some of teh warning may be ignored. Yet, pl;ease pay close attention to them as some of them may cause unexpected beavior.

In [ ]:
# First we install REWET
!pip install rewet



## Input

To run the example, we need to prepare the input. The easiest way to give input is to create a JSON file. Damage data, also part of the input, are prepared as a spreadsheet as the tabulated format of the data make it easier to use spreadsheet.


## City

The network used is Net3, which comes as a testbed for testing EPANET package.
We Assume that the network is providing water to people as is. We want to run the simulation for 5 days. The units in REWET is SI which means that time unit is in seconds. As a result simulation time would be 432,000 s.


## Damage
 We assume that the damage happens in hour 2 of simulation. It is assuemd that the damage modeling for break (aka material) is Cast Iron. The damages in pipes can be seen the following image.

 ![Exampel 1 damage](https://snaeimi.github.io/REWET/_static/Example_1.jpg)

he list of damage are as follows:


|   time |   pipe_id |   damage_loc | type   | Material   |
|-------:|----------:|-------------:|:-------|:-----------|
|   7200 |       131 |         0.7  | leak   | CI         |
|   7200 |       131 |         0.8  | break  | CI         |
|   7200 |       204 |         0.9  | leak   | CI         |
|   7200 |       204 |         0.8  | break  | CI         |
|   7200 |       204 |         0.5  | leak   | CI         |
|   7200 |       225 |         0.5  | leak   | CI         |
|   7200 |       177 |         0.5  | leak   | CI         |
|   7200 |       153 |         0.15 | break  | CI         |
|   7200 |       147 |         0.15 | break  | CI         |


One pumps is damaged at hour 2 and get back to the serive 10 hours later.

| Pump_ID | Time           | Restore_time           |
|---------|----------------|------------------------|
| 10      | 7200           | 36000                  |


Two tanks are damaged at hour 2 and get back to teh serive 20 hours later.

| Tank_ID | Time | Restore_time |
|---------|------|--------------|
|    1    | 7200 |    72000     |


The node damages happen in nodes 109, 121, 149, and 199. Number of damages and the node pipe length are used to calculaet the median repair rate in the node (refer to the manual for more information). The input data is a table as shown here:

| Time | Node Name | Number of Damages | Node Pipe Length |
|------|------------|-------------------|------------------|
| 7200 | 109        | 1                 | 10               |
| 7200 | 121        | 4                 | 8                |
| 7200 | 149        | 2                 | 6.5              |
| 7200 | 199        | 1                 | 6.8              |

You enter the data as shown in tables, each at one excel file and save them all in one location. The next step is making list (aka scenario table) excel file. for thsi example, we have only one damage scenario, so there will be only one row (row row for each scenario). The information included are scenario name, the name of damage files for pipes, pumpus, tanks and nodes, and teh probability of the scearnio. Since we have only one scenario here, the probability is selected as 1.

| Scenario Name | Pipe Damage               | Nodal Damage              | Pump Damage              | Tank Damage              | Probability |
|---------------|---------------------------|---------------------------|--------------------------|--------------------------|-------------|
| Net3-Example  | Net3-pipe-damage.xlsx     | Net3-node-damage.xlsx     | Net3-PumpDamage.xlsx     | Net3_tank_damage.xlsx    | 1           |




In [ ]:
import rewet
import json
import tempfile
from pathlib import Path
import pandas as pd

# make dictionary that keep all the input data
input_data = {}

# make sure you have mounted the drive
input_data["WN_INP"] = "/content/drive/MyDrive/REWET Examples/Net3/Net3.inp"
# run time is set to 5 days
input_data["RUN_TIME"] = 5 * 24 * 3600
# request a temporary directory from the platform. You can use an existing
# location for the temporary file locations too.
input_data["temp_directory"] = tempfile.gettempdir()
# set a location for the result
input_data["result_directory"] = "/content/drive/MyDrive/Result"

input_data["pipe_damage_file_list"] = \
 "/content/drive/MyDrive/REWET Examples/Net3/list.xlsx"
input_data["pipe_damage_file_directory"] = \
"/content/drive/MyDrive/REWET Examples/Net3/Damages"
input_data["Restortion_config_file"] = \
"/content/drive/MyDrive/REWET Examples/Net3/config.txt"





## Save the input data

In [ ]:
# save the input data
input_data_location = "./input.json"
with open(input_data_location, "wt") as f:
  json.dump(input_data, f)

print(f" the input data is saved in {input_data_location}")

## Run REWET

In [ ]:
# make an instance of Starter
start = rewet.initial.Starter()

# run the simulation with the given path
start.run("./input.json")


## Results

Now taht you have successfully ran REWET, you can load results that are saved in the saved location. Then you can get raw data and postporcess it or used functions that are available in REWET post-processing module, Result.

In [ ]:
# load the result. The project.prj is always created as part of the output.
# it also serves as a input file.
result = rewet.Result(Path(input_data["result_directory"])/"project.prj")


### Crew Report

You can run functions provided by REWET to get the detailed and averaged results (daily).

In [ ]:
# get detailed crew report result for RepairAgent
crew_detailed_RepairAgent = result.getCrewCompleteStatusReport(
                                                  "Net3-Example", "RepairAgent")

print(crew_detailed_RepairAgent.head())



In [ ]:
# get averaged crew report for RepairAgent
crew_complete_RepairAgent = result.averageOverDaysCrewTotalReport(
                                                      crew_detailed_RepairAgent)

print(crew_complete_RepairAgent.head())


### Demand saiesfaction

Now lets get the Water Delivery, and Quantity BBasic Service Category.

In [ ]:
delivery_map = result.getOutageTimeGeoPandas_4("Net3-Example",
                                               LOS='DL',
                                               iConsider_leak=False,
                                               leak_ratio=0,
                                               consistency_time_window=0
                                              )

result.convertTimeSecondToDay(delivery_map, "restoration_time")
delivery_map.plot(column="restoration_time", legend=True)


quality_map = result.getOutageTimeGeoPandas_4("Net3-Example",
                                               LOS='QN',
                                               iConsider_leak=False,
                                               leak_ratio=0,
                                               consistency_time_window=0
                                               )

result.convertTimeSecondToDay(quality_map, "restoration_time")
quality_map.plot(column="restoration_time", legend=True)



# save shapefile
delivery_map.to_file(Path(input_data["result_directory"])/"Delivery")
quality_map.to_file(Path(input_data["result_directory"])/"Quality")

In [ ]:
delivery_curve = result.getDLIndexPopulation_4("Net3-Example", ratio=True) * 100
delivery_curve.index = delivery_curve.index / 3600 / 24
delivery_curve.plot()